# GeometricAnnotationErrors - EM Algorithm
### July 2021

---

Used to run the EM algorithm **AFTER** downloading the source data and running `preprocess.py` to create training, validation, and testing tensors.

Test results and plots are output to a indexed sub-folder (like 'em_test_00/') under the directory specified by `RESULTS_DIR` in `config.py`. If you do not have a `config.py` with this variable in your root directory, use `setup.py` to initialize your environment. Feel free to set the test output root directory to any folder that does not require root permissions to access. 

In [ ]:
""" Load Filepaths """
from config import INPUT_DATA_DIR, TENSOR_DIR, RESULTS_DIR
data_path = TENSOR_DIR
source_path = INPUT_DATA_DIR
out_root_dir = RESULTS_DIR

# Output level - Higher levels include lower outputs
# 0: 
# - info markdown, 
# - step-wise graph for train/val/test F1, shapefile IoU 
# 1: 
# - model plots (each step) 
# - model weights (each step)
# - updated annotation shapefiles
# 2+: 
# - Predicted class map rasters
# - annotation rasters, 
# - model reports in text
# - Training history for each step
# - Tensorboard for every step
OUTPUT_LEVEL = 0

import os

# Module Imports
import tensorflow as tf
import numpy as np
import rasterio as rio
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely.geometry as shp
from tensorflow.keras.optimizers import Adam

# Lib imports
import lib.Doc_Tools as doc
import lib.GeoTools as gt
import lib.Tiling as tile
import lib.K_Tools as kt

from lib import *
from lib.envtools import gettime, lr_schedule
from lib.Doc_Tools import print_s

### Model Training Hyperparameters
BATCH_SIZE = 32 
EPOCHS = 50

# Seed environment
tf.random.set_seed(2001)
np.random.seed(2001)

### Create Folder to store test results
test_idx, test_dir = doc.InitTest(out_root_dir)

# Open Raster Imagery, 
train_raster = rio.open(os.path.join(source_path, 'train_raster.tif'))

# Open Label shapefiles
refined_labels  = gpd.read_file(os.path.join(source_path, 'refined_labels.shp'))
refined_labels.to_crs(train_raster.crs, inplace=True)

imperfect_labels = gpd.read_file(os.path.join(source_path, 'imperfect_labels.shp'))
imperfect_labels.to_crs(train_raster.crs, inplace=True)
print_s(None, "Successfully loaded rasters and shapefiles data.")

### Loading Tensors and tile offsets
X_train = np.load(os.path.join(data_path, 'X_train.npy'))
Y_train = np.load(os.path.join(data_path, 'Y_train.npy'))
X_val = np.load(os.path.join(data_path, 'X_val.npy'))
Y_val = np.load(os.path.join(data_path, 'Y_val.npy'))
X_test = np.load(os.path.join(data_path, 'X_test.npy'))
Y_test = np.load(os.path.join(data_path, 'Y_test.npy'))
train_offsets_fp = os.path.join(data_path, 'train_offsets.csv')
val_offsets_fp = os.path.join(data_path, 'val_offsets.csv')
print_s(None, "Successfully loaded tensors.")

## Baseline Model Training and Evaluation
First, the UNet is trained on the source data generated by `preprocess.py`. The results of this training can be found under the 'baseline' sub-folder in the test's output.

In [ ]:
# Prepare Baseline Folder
base_folder = os.path.join(test_dir, 'baseline')
if not os.path.exists(base_folder): os.mkdir(base_folder)
    
# Prepare optimizer and callbacks (including weight output)
optimizer = Adam(lr=0.1, epsilon=1e-8, decay=1e-5)
base_callbacks = kt.SetCallbacks(weights_out=os.path.join(base_folder, 'BaselineWeights.h5'),
                                 tensorboard_path=os.path.join(base_folder, 'tensorboard'),
                                 output_level=OUTPUT_LEVEL)

# Select and Build Model
model = kt.Get_Model('UNET')
model.compile(optimizer=optimizer, 
              loss = kt.dice_coef_loss, 
              metrics=[kt.dice_coef,'accuracy', kt.f1_score])

# Train Model
baseline_results = model.fit(X_train, 
                             Y_train, 
                             validation_data=(X_val, Y_val), 
                             shuffle=True, 
                             batch_size=BATCH_SIZE, 
                             epochs=EPOCHS, 
                             callbacks=base_callbacks)

# Save Results as plot (2+)
if (OUTPUT_LEVEL >= 1):
    doc.plot_history(baseline_results, 
                     test_dir=base_folder, 
                     config_idx='base')

# Evaluate Baseline Model and display results on all three sets 
print("Baseline Model Preformance:")
train_rpt = kt.ModelReport(X_train, Y_train, model, 'Training', print_report=True)
val_rpt = kt.ModelReport(X_val, Y_val, model, 'Validation', print_report=True)
test_rpt = kt.ModelReport(X_test, Y_test, model, 'Testing', print_report=True)

print_s(None, "Completed Baseline UNet Training.")

### Configure Annotator Class
The following class is used to  update the source annotation based on the UNet's predicted class map output. For time cost, all candidate geometries are preemptively generated with minimal metadata. 

This preloading process may take some time depending on your hardware. To reduce the memory cost, you can try reducing the total number of candidate geometries with the `pairs` attribute.

In [ ]:
annotator = Dynamic_Preloading_Annotator(
    pairs=15,       # pairs: Number of candidates generated on either side 
    off_dist=1.5,   # of each shiftable vertex in the imperfect labels 
    interval=10,    
    min_p=1e-02,
    L=0.02,                
    weight_buffer=2
)
# Preload Candidate Data 
initial_pmap = kt.Get_Pmap(source_raster=train_raster, pmodel=model)
all_data = annotator.preload_candidates(imperfect_labels, initial_pmap)

## EM Iteration
The following block loops through fourteen steps of the EM algorithm. During the running, you'll be able to follow the progress of the UNet model's training and the annotation refinement through the cell's output. The same output will be saved to an indexed test folder under the directory specified by `RESULTS_DIR` in `config.py`.

In [ ]:
# Begin Iteration
print_s(None, "Beginning Iteration, Target steps: 14")
for em_idx in range(0, 14): 
    # EM Step Initialization
    # ----------------------
    emfolder = os.path.join(test_dir, f'step_{em_idx:02}')
    if not os.path.exists(emfolder): os.mkdir(emfolder)
    print_s(em_idx, "Started EM Step")
    
    # Set first-iteration variables
    if (em_idx == 0):
        ### Evaluate original Shapefile Precision
        source_iou = gt.gdf_iou(refined_labels, imperfect_labels)
        prev_iou = source_iou
        
        # Stores EM results
        em_dict = {
            'Name': ['Base'],
            'Test_Data': [test_rpt],
            'Train_Data': [train_rpt],
            'Val_Data': [val_rpt],
            'Line_IoU': [np.round((source_iou*100), 2)],
            'Epochs': [len(baseline_results.history['accuracy'])]
        }
        # Stores best values for this test
        top_f1, top_f1_idx = 0, 0
        top_iou, top_iou_idx = 0, 0
        
        buff_dist = 4
        pmap_fp = os.path.join(emfolder, 'pmap_baseline.tif')
    else:
        buff_dist = 2
        pmap_fp = os.path.join(emfolder, f'pmap_{em_idx:02}.tif')
    
    
    # Override pmap output for output_level
    
    # Update Annotation
    # -----------------
    # Get predicted class map 
    predicted_class_map = kt.Get_Pmap(train_raster, model, pmap_fp)
    print_s(em_idx, "Generated Predicted Class Map from UNet model.")

    # Update annotation
    annotation_fp = os.path.join(emfolder, f'annotation_{em_idx:02}.shp')
    new_annotation = annotator.update_gdf_from_preload(all_data, class_map=predicted_class_map, out_path=annotation_fp) 
    print_s(em_idx, "Created New Annotation.")
    
    # Save iou for this annotation.
    anno_iou = gt.gdf_iou(refined_labels, new_annotation)


    # Create new Label Tensors
    # ------------------------
    # Buffer
    buff_anno = gt.gdf_buffer(new_annotation, buff_dist=buff_dist, flatten=True)
    
    if (OUTPUT_LEVEL >= 2):
        anno_raster_fp = os.path.join(emfolder, f'rasterized_annotation_{em_idx:02}.tif')
    else: anno_raster_fp = None
        
    # Rasterize New Labels
    anno_raster = gt.GDF_Rasterize(buff_anno, train_raster, out_path=anno_raster_fp)
    
    # Read Y_train, Y_val tensors from rasterized label
    Y_train = tile.ResampleTiles(anno_raster, train_offsets_fp)
    Y_val = tile.ResampleTiles(anno_raster, val_offsets_fp)
    
    # Upsample label tensors to match shape
    Y_train = tile.AugmentImages(Y_train, h_flip=False, v_flip=True, rotate=True)
    Y_val = tile.AugmentImages(Y_val, h_flip=False, v_flip=True, rotate=True)
    print_s(em_idx, f"Created Y_train {Y_train.shape} and Y_val {Y_val.shape}.")
    
    if (OUTPUT_LEVEL >= 3):
        # Save udpated label tenors
        np.save(os.path.join(emfolder, f'Y_train_{em_idx:02}'), Y_train)
        np.save(os.path.join(emfolder, f'Y_val_{em_idx:02}'), Y_val)
    
    # Re-Train UNet 
    # -------------
    # Prepare optimizer, load callbacks 
    em_optimizer = Adam(lr=lr_schedule(em_idx), epsilon=1e-8, decay=1e-5)
    callbacks = kt.SetCallbacks(weights_out=os.path.join(emfolder, f'unet_weights_{em_idx:02}.h5'), 
                                 tensorboard_path=os.path.join(emfolder, f'tensorboard_{em_idx:02}'),
                                 output_level=OUTPUT_LEVEL)
    
    # Re-compile and Train Model
    model = kt.Get_Model('UNET')
    model.compile(optimizer=em_optimizer, 
                  loss=kt.dice_coef_loss, 
                  metrics=[kt.dice_coef,'accuracy', kt.f1_score])
    training_history = model.fit(X_train, 
                                 Y_train, 
                                 validation_data=(X_val, Y_val), 
                                 shuffle=True, 
                                 batch_size=BATCH_SIZE, 
                                 epochs=EPOCHS, 
                                 callbacks=callbacks,
                                 verbose=0)
    print_s(em_idx, "Completed model Training.")
    
    # Save Training History 
    if (OUTPUT_LEVEL >= 1):
        doc.plot_history(training_history, test_dir=emfolder, config_idx=em_idx)
    
    # Evaluate Model on train, validation, and test sets 
    if (OUTPUT_LEVEL >= 2):
        # If last step saved to markdown, close the previous file 
        if hist_md: hist_md.close()
        hist_markdown_fp = os.path.join(emfolder, f'history_{em_idx:02}.md')
        hist_md = open(hist_markdown_fp, 'w+')
    else: hist_md = None
    
    test_rpt = kt.ModelReport(X_test, Y_test, model, "Testing", index=(em_idx+1), report_md=hist_md)
    train_rpt = kt.ModelReport(X_train, Y_train, model, "Training", index=(em_idx+1), report_md=hist_md)
    val_rpt = kt.ModelReport(X_val, Y_val, model, "Validation", index=(em_idx+1), report_md=hist_md)
        
    # Summarize EM Step
    # -----------------
    # Record performance
    em_dict['Name'].append(f'Step {em_idx:02}')
    em_dict['Line_IoU'].append(np.round((anno_iou*100), 2))
    em_dict['Epochs'].append(len(training_history.history['accuracy']))
    em_dict['Test_Data'].append(test_rpt)
    em_dict['Train_Data'].append(train_rpt)
    em_dict['Val_Data'].append(val_rpt)
    
    # Update top scores for F1 and Annotation IoU
    if test_rpt['F1_Score'] > top_f1:
        top_f1 = test_rpt['F1_Score']
        top_f1_idx = em_idx
        top_f1_txt = "(TOP)"
    else: top_f1_txt = ""
    if np.round((anno_iou*100), 2) > top_iou:
        top_iou = np.round((anno_iou*100), 2)
        top_iou_idx = em_idx
        top_iou_txt = "(TOP)"
    else: top_iou_txt = ""
    
    # Print step data
    print(f"\nEM Step {em_idx:02} Complete. [{gettime('%b %d | %I:%M:%S%p')}]")
    print(f'- Annotation IoU:     {(anno_iou*100):.2f}')
    print(f'  - Source Improvement: {((anno_iou-source_iou)*100):+.2f}')
    print(f'  - Step Improvement:   {((anno_iou-prev_iou)*100):+.2f}')
    print('- Model Performance: (trained on new labels)')
    print('  - Training Report (Step {em_idx:02}):')
    doc.PrintReport(train_rpt)
    print('  - Validation Report (Step {em_idx:02}):')
    doc.PrintReport(val_rpt)
    print('  - Testing Report (Step {em_idx:02}):')
    doc.PrintReport(test_rpt)
    print("----------------------------------\n\n")
    
    # Increase iterator and save previous precision for step_delta
    prev_iou = anno_iou

## **Results**
The following code is not mandatory, though it will help to understand the improvements made to the source annotation through the EM pipeline.

### **Results**: Plot F1, IoU
The following block converts the model evaluation data saved in `em_dict` into pretty matplotlib plots. These are saved in the results folder and output below.

In [ ]:
# Reformat EM Results and plot metrics by EM step
model_dict = doc.get_model_dict(em_dict=em_dict)
fig, axs = plt.subplots(2, 2, sharex=True, figsize=(16,10))
doc.plot_axis(ax=axs[0,0], # Testing F1
              data=model_dict['Test_Data']['F1_Score']*100, 
              name='Testing F1', 
              color_char='r', 
              symbol_char='s', 
              y_off=2)
doc.plot_axis(ax=axs[0,1], # Training F1
              data=model_dict['Train_Data']['F1_Score']*100, 
              name='Training F1', 
              color_char='g', 
              symbol_char='^', 
              y_off=2)
doc.plot_axis(ax=axs[1,0], # Validation F1
              data=model_dict['Val_Data']['F1_Score']*100, 
              name='Validation F1', 
              color_char='c', 
              symbol_char='^', 
              y_off=2)
doc.plot_axis(ax=axs[1,1], #Annotation IoU
              data=em_dict['Line_IoU'], 
              name='Line IoU', 
              color_char='m', 
              x_label='EM Step')
    
## Title and show, and save figure
fig.suptitle("EM Test {:02}".format(test_idx))
fig_path = os.path.join(test_dir, 'results_plot_{:02}.png'.format(test_idx))
fig.savefig(fig_path)
fig.show()

### **Results**: Save Info to Markdown
Most of the essential information printed for the EM pipeline is saved to a markdown file below. It contains model performance, annotation metrics, and  

In [ ]:
### Write test data to markdown
markdown_fp = os.path.join(test_dir, 'info_{:02}.md'.format(test_idx))
md = open(markdown_fp, 'w+')

# Header
md.write("# Geometric Annotation Errors - EM Test {:02}\n".format(test_idx))
md.write(f"### {gettime('%b %d | %I:%M:%S%p')}\n")
md.write("\n---\n\n")
# Results Section
md.write("## **Results**:\n\n")
top_f1 = np.round((top_f1*100), 2)
source_f1 = np.round((em_dict['Test_Data'][0]['F1_Score']*100), 2)
source_iou = np.round(source_iou*100, 2)
md.write("### Top Values:\n")
md.write(f" - Testing F1 Score: **{top_f1}** (`{top_f1-source_f1}`) - Step {top_f1_idx}\n")
md.write(f" - Annotation IoU: **{top_iou}** (`{top_iou-source_iou}`) - Step {top_iou_idx}\n\n")

# Write results by EM step
md.write("### EM Iteration:\n")
md.write("Step | Anno IoU | F1 | Epochs | LR | Train | Update\n")
md.write("---- | -------- | -- | ------ | -- | ----- | ------\n")
for idx in range(14):
    if idx == 0:
        md.write("{} | {} | {} | {} | {} | {} | {}\n".format(em_dict['Name'][idx], em_dict['Line_IoU'][idx], em_dict['Test_Data'][idx]['F1_Score'], em_dict['Epochs'][idx], em_dict['LR'][idx], em_dict['Training_Time'][idx], em_dict['Update_Time'][idx]))
    else:
        md.write("{} | {} (`{:+.2f}`) | {} | {} | {} | {} | {}\n".format(em_dict['Name'][idx], em_dict['Line_IoU'][idx], (em_dict['Line_IoU'][idx] - em_dict['Line_IoU'][0]), em_dict['Test_Data'][idx]['F1_Score'], em_dict['Epochs'][idx], em_dict['LR'][idx], em_dict['Training_Time'][idx], em_dict['Update_Time'][idx]))
md.write("\n\n</br>\n\n")
md.write("### Model Performance:\n\n")
md.write("Step | Test F1 | Test (FP, FN) | Train F1 | Train (FP, FN) | Val F1 | Val (FP, FN) \n")
md.write("---- | ------- | ------------- | -------- | -------------- | ------ | ------------ \n")
for idx in range(0, 14):
    # Create a string to hold this row's data
    row_string = f"{em_dict['Name'][idx]} | "
    for key in ['Test_Data', 'Train_Data', 'Val_Data']:
        row_string += "{:.2f} | ({:.2e}, {:.2e}) | ".format(em_dict[key][idx]['F1_Score']*100, em_dict[key][idx]['False_Positives'], em_dict[key][idx]['False_Negatives'])
    md.write(row_string)
md.close()
print_s(None, f"Results written to {markdown_fp}.")